<a href="https://colab.research.google.com/github/mulak1/Crew_AI_Fine_Tuned_Comedian_w_HeyGen/blob/main/Crew_AI_Fine_Tuned_Comedian_w_HeyGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# STEP 1: Setup OpenAI Client (with proper import)
import os
import openai
from google.colab import userdata

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY2')
    if not OPENAI_API_KEY:
        raise ValueError("❌ OPENAI_API_KEY2 not found in secrets.")

    openai_client = openai.OpenAI(api_key=OPENAI_API_KEY)
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY  # Optional, for LangChain/CrewAI compatibility
    print("✅ OpenAI client initialized.")

    # Set default model
    MODEL_NAME = "gpt-3.5-turbo"
    print(f"📦 Using model: {MODEL_NAME}")

except Exception as e:
    print(f"❌ Failed to initialize OpenAI: {e}")
    raise SystemExit("🛑 Please ensure OPENAI_API_KEY2 is in Colab secrets.")


✅ OpenAI client initialized.
📦 Using model: gpt-3.5-turbo


In [1]:
# STEP 2 (Revised): Load smaller model (phi-1_5) for CPU fine-tuning

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import prepare_model_for_kbit_training

# Check CPU mode
if torch.cuda.is_available():
    print(f"⚠️ GPU available: {torch.cuda.get_device_name(0)} — but using CPU mode.")
else:
    print("💻 Running on CPU (phi-1_5 should fit fine).")

# Use smaller model
model_name = "microsoft/phi-1_5"

# Load model
try:
    print(f"⏳ Loading model: {model_name}")
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True
    )
    model.config.use_cache = False
    print("✅ Model loaded successfully.")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    raise SystemExit("🛑 Failed to load smaller model.")

# Load tokenizer
try:
    print("⏳ Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print("⚠️ No pad_token found. Using eos_token as pad_token.")

    tokenizer.padding_side = "right"
    print("✅ Tokenizer loaded.")
except Exception as e:
    print(f"❌ Tokenizer loading error: {e}")
    raise SystemExit("🛑 Failed to load tokenizer.")

# Prep model for LoRA PEFT (still needed)
try:
    model = prepare_model_for_kbit_training(model)
    print("✅ Model prepared for PEFT fine-tuning.")
except Exception as e:
    print(f"⚠️ Could not apply PEFT prep: {e}")


💻 Running on CPU (phi-1_5 should fit fine).
⏳ Loading model: microsoft/phi-1_5


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

⚠️ No pad_token found. Using eos_token as pad_token.
✅ Tokenizer loaded.
✅ Model prepared for PEFT fine-tuning.


In [3]:
# Install Hugging Face Datasets + PEFT dependencies (if not already installed)
!pip install -q datasets peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 900.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
# STEP 3: LoRA + Dataset + Simple Prompt Formatting (phi-1_5 friendly)



import os
import json
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

# --- LoRA Configuration ---
print("🧠 Configuring LoRA...")
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

try:
    model = get_peft_model(model, lora_config)
    print("✅ PEFT model wrapped with LoRA.")
    model.print_trainable_parameters()
except Exception as e:
    print(f"❌ Error applying LoRA: {e}")
    raise SystemExit("🛑 Could not configure LoRA.")

# --- Create Dummy Dataset ---
dummy_data = [
    {
        "input": "Cats using Zoom",
        "output": "My cat joined my Zoom call and got promoted. Now she’s my manager."
    },
    {
        "input": "AI making dating profiles",
        "output": "AI wrote my dating profile. Now I’m getting matched with robots."
    }
]

dataset_path = "topical_jokes_dataset_openai.jsonl"
with open(dataset_path, "w", encoding="utf-8") as f:
    for example in dummy_data:
        f.write(json.dumps(example) + "\n")

print(f"✅ Dummy dataset saved as '{dataset_path}'")

# --- Load Dataset ---
try:
    dataset = load_dataset("json", data_files=dataset_path, split="train")
    print(f"✅ Loaded dataset with {len(dataset)} examples.")
    print(dataset[0])
except Exception as e:
    print(f"❌ Failed to load dataset: {e}")
    raise SystemExit("🛑 Dataset loading failed.")

# --- Simple Prompt Formatter (no chat template) ---
def format_example(example):
    topic = example.get("input", "")
    joke = example.get("output", "")
    try:
        formatted = f"User: Generate a short, funny stand-up style joke about: {topic}\nAssistant: {joke}"
        return {"text": formatted}
    except Exception as e:
        print(f"⚠️ Error formatting: {e}")
        return {"text": None}

print("✨ Formatting dataset for training (non-chat format)...")
dataset = dataset.map(format_example)
dataset = dataset.filter(lambda x: x["text"] is not None and len(x["text"]) > 0)

print(f"✅ Dataset formatted. Final usable size: {len(dataset)}")
print("\n📝 Example formatted text:")
print(dataset[0]["text"])



🧠 Configuring LoRA...
✅ PEFT model wrapped with LoRA.
trainable params: 14,155,776 || all params: 1,432,426,496 || trainable%: 0.9882
✅ Dummy dataset saved as 'topical_jokes_dataset_openai.jsonl'


Generating train split: 0 examples [00:00, ? examples/s]

✅ Loaded dataset with 2 examples.
{'input': 'Cats using Zoom', 'output': 'My cat joined my Zoom call and got promoted. Now she’s my manager.'}
✨ Formatting dataset for training (non-chat format)...


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2 [00:00<?, ? examples/s]

✅ Dataset formatted. Final usable size: 2

📝 Example formatted text:
User: Generate a short, funny stand-up style joke about: Cats using Zoom
Assistant: My cat joined my Zoom call and got promoted. Now she’s my manager.


In [12]:
# STEP 4: Fine-Tune using SFTTrainer (CPU-safe setup)

!pip install -q trl

from transformers import TrainingArguments
from trl import SFTTrainer

# --- Training Args ---
output_dir = "./phi1_5_joke_adapter"
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,  # Increase if more data
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    logging_steps=1,
    save_strategy="epoch",
    max_grad_norm=1.0,
    lr_scheduler_type="linear",
    bf16=False,
    fp16=False,
    gradient_checkpointing=False,
    report_to="none",  # No wandb
)

print("✅ Training args set.")

# --- Start Training ---
print("\n🚀 Launching fine-tuning with SFTTrainer...")

try:
    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        args=training_args,
        peft_config=lora_config
    )

    trainer.train()
    print("\n🏁 Fine-tuning complete!")

    # --- Save everything ---
    print(f"💾 Saving LoRA adapter to: {output_dir}")
    trainer.save_model()
    trainer.save_state()

except Exception as e:
    print(f"❌ Training failed: {e}")
    import traceback
    traceback.print_exc()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 5.1 MB/s eta 0:00:00
✅ Training args set.

🚀 Launching fine-tuning with SFTTrainer...


Converting train dataset to ChatML:   0%|          | 0/2 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,4.042400
2,3.629500
3,3.323400



🏁 Fine-tuning complete!
💾 Saving LoRA adapter to: ./phi1_5_joke_adapter


In [2]:
# STEP 5.1: Safe Reload of Base + LoRA Model on CPU (Low RAM)

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# --- Base model + adapter paths
base_model_name = "microsoft/phi-1_5"
adapter_path = "./phi1_5_joke_adapter"

# --- Load tokenizer
print("⏳ Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("⚠️ No pad_token found, using eos_token instead.")
tokenizer.padding_side = "right"
print("✅ Tokenizer ready.")

# --- Load base model in float16 for RAM efficiency
print("⏳ Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
)
base_model.config.use_cache = False
print("✅ Base model loaded.")

# --- Attach LoRA adapter
print(f"🔧 Loading LoRA adapter from {adapter_path}...")
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()
print("✅ Fine-tuned model ready for inference.")


⏳ Loading tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


⚠️ No pad_token found, using eos_token instead.
✅ Tokenizer ready.
⏳ Loading base model...
✅ Base model loaded.
🔧 Loading LoRA adapter from ./phi1_5_joke_adapter...
✅ Fine-tuned model ready for inference.


/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.l

In [3]:
# STEP 5.2: Generate a Joke from Your Fine-Tuned Phi-1.5 + LoRA

def generate_joke(topic: str):
    prompt = f"User: Generate a short, funny stand-up style joke about: {topic}\nAssistant:"
    inputs = tokenizer(prompt, return_tensors="pt")

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50,
        temperature=0.8,
        do_sample=True,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id
    )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    joke = output_text[len(prompt):].strip()
    return joke

# 🔁 Run a test
test_topic = "AI trying to understand Gen Z slang"
print(f"🎤 Topic: {test_topic}")
print("🧠 Generating...")

joke = generate_joke(test_topic)
print(f"\n😂 Joke:\n{joke}")


🎤 Topic: AI trying to understand Gen Z slang
🧠 Generating...

😂 Joke:
"AI, have you ever tried to use Gen Z slang in your calculations? It's like trying to learn a whole new language!"

5. Exercise: Write a short script for an animated short film about a group of friends who embark on


In [4]:
# STEP 6.1: Install CrewAI and set up OpenAI key (still needed for orchestration)

!pip install -q crewai langchain-openai crewai[tools]

from crewai import Agent, Task, Crew, Process
from google.colab import userdata
import os

# --- Set OpenAI key (for CrewAI orchestration only)
try:
    OPENAI_API_KEY = userdata.get("OPENAI_API_KEY2")
    if not OPENAI_API_KEY:
        raise ValueError("OPENAI_API_KEY2 missing from Colab secrets")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"  # Or any small GPT
    print("✅ CrewAI API key and model set.")
except Exception as e:
    print(f"❌ Error: {e}")
    raise SystemExit("🛑 Cannot continue without OpenAI key for CrewAI")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.5 MB/s eta 0:00

In [6]:
# STEP 6.2 (REVISED): Robust import of `@tool` decorator

import importlib

# Try multiple paths for importing the tool decorator
tool = None
try:
    from crewai_tools import tool
    print("✅ Loaded @tool from crewai_tools")
except ImportError:
    try:
        from crewai import tool
        print("✅ Loaded @tool from crewai")
    except ImportError:
        try:
            from crewai.tools import tool
            print("✅ Loaded @tool from crewai.tools")
        except ImportError as e:
            print("❌ Could not import `tool` decorator from any known path.")
            raise e

# Define your tool
@tool("Fine-Tuned Joke Generator")
def joke_generation_tool(topic: str) -> str:
    """Generates a short, funny stand-up style joke using a finetuned Phi-1.5 model."""
    try:
        if not topic:
            return "Error: No topic provided."

        joke = generate_joke(topic)
        if not joke or "Error" in joke:
            return "Hmm, I couldn't come up with a joke this time. Try a new topic!"

        return joke
    except Exception as e:
        return f"Error while generating joke: {e}"


✅ Loaded @tool from crewai.tools


In [8]:
# STEP 6.3: Define CrewAI Agent and Task for Joke Generation

from crewai import Agent, Task, Crew, Process

# --- Agent (the comedian)
joke_writer_agent = Agent(
    role="Stand-up Comedian",
    goal="Generate short, funny, topical jokes using the joke tool only",
    backstory=(
        "You are Chuckles, an AI comedian trained in observational humor and absurdity. "
        "Your only method of creating jokes is the 'Fine-Tuned Joke Generator' tool — "
        "you may not improvise or make jokes on your own. Trust the tool!"
    ),
    verbose=True,
    allow_delegation=False,
    tools=[joke_generation_tool]  # From previous step
)

# --- Task
joke_task = Task(
    description=(
        "You will be given a topic in triple backticks: ```{topic}```. "
        "Use the 'Fine-Tuned Joke Generator' tool to generate one joke based on this topic. "
        "Your output must be only the joke — no explanation, intro, or commentary."
    ),
    expected_output="One stand-up style joke related to the topic.",
    agent=joke_writer_agent
)

print("✅ Agent and Task defined.")


✅ Agent and Task defined.


In [9]:
# STEP 6.4: Kickoff CrewAI with a sample topic

# Create the Crew (assigning agent + task)
joke_crew = Crew(
    agents=[joke_writer_agent],
    tasks=[joke_task],
    process=Process.sequential,
    verbose=True
)

# --- Test Topic
test_topic = "Smart homes that talk back"
print(f"🚀 Sending topic to Crew: '{test_topic}'")

# Run the crew with the topic
result = joke_crew.kickoff(inputs={"topic": test_topic})

# Display result
print("\n🎤 Final Joke:")
print(result)


🚀 Sending topic to Crew: 'Smart homes that talk back'


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c49ed43f-510a-46f6-bf93-6dd6f2ea7162                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 8848705e-2029-4c3f-9466-2097f6ca40ff
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 8848705e-2029-4c3f-9466-2097f6ca40ff
       Status: Executing Task...
    └── 🤖 Agent: Stand-up Comedian
            Status: In Progress

# Agent: Stand-up Comedian
## Task: You will be given a topic in triple backticks: ```Smart homes that talk back```. Use the 'Fine-Tuned Joke Generator' tool to generate one joke based on this topic. Your output must be only the joke — no explanation, intro, or commentary.


...

🚀 Crew: crew
└── 📋 Task: 8848705e-2029-4c3f-9466-2097f6ca40ff
       Status: Executing Task...
    └── 🤖 Agent: Stand-up Comedian
            Status: ✅ Completed
        └── 🔧 Used Fine-Tuned Joke Generator (1)

🚀 Crew: crew
└── 📋 Task: 8848705e-2029-4c3f-9466-2097f6ca40ff
       Assigned to: Stand-up Comedian
       Status: ✅ Completed
    └── 🤖 Agent: Stand-up Comedian
            Status: ✅ Completed
        └── 🔧 Used Fine-Tuned Joke Generator (1)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8848705e-2029-4c3f-9466-2097f6ca40ff                                                                     │
│  Agent: Stand-up Comedian                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c49ed43f-510a-46f6-bf93-6dd6f2ea7162                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


🎤 Final Joke:
Why did the smart thermostat go to therapy? Because it felt a little overwhelmed by all the noise from the voice-activated speakers!


In [10]:
# STEP 7.1: Configure HeyGen

import requests
import time

# Load from Colab secrets
from google.colab import userdata

try:
    HEYGEN_API_KEY = userdata.get('HEYGEN_API_KEY')
    if not HEYGEN_API_KEY:
        raise ValueError("Missing HEYGEN_API_KEY in Colab secrets.")
    print("✅ HeyGen API Key loaded.")
except Exception as e:
    print(f"❌ {e}")
    raise SystemExit("🛑 Add HEYGEN_API_KEY to Colab secrets to continue.")

# --- Basic Config ---
DEFAULT_AVATAR_ID = "Daisy-inskirt-20220818"  # Replace with your preferred one
DEFAULT_VOICE_ID = "2d5b0e6cf36f460aa7fc47e3eee4ba54"  # Replace with compatible voice ID

HEYGEN_GENERATE_URL = "https://api.heygen.com/v2/video/generate"
HEYGEN_STATUS_URL = "https://api.heygen.com/v1/video_status.get"


✅ HeyGen API Key loaded.


In [11]:
# STEP 7.2: Define function to generate HeyGen video for the joke

def generate_heygen_video(joke_text: str) -> str | None:
    if not HEYGEN_API_KEY or not joke_text:
        print("❌ Missing HeyGen API key or joke text.")
        return None

    headers = {
        "X-Api-Key": HEYGEN_API_KEY,
        "Content-Type": "application/json"
    }

    payload = {
        "video_inputs": [
            {
                "character": {
                    "type": "avatar",
                    "avatar_id": DEFAULT_AVATAR_ID,
                    "avatar_style": "normal"
                },
                "voice": {
                    "type": "text",
                    "input_text": joke_text,
                    "voice_id": DEFAULT_VOICE_ID
                }
            }
        ],
        "dimension": {
            "width": 1280,
            "height": 720
        },
        "test": False
    }

    try:
        print("📤 Sending joke to HeyGen...")
        response = requests.post(HEYGEN_GENERATE_URL, headers=headers, json=payload, timeout=30)
        response.raise_for_status()
        data = response.json()

        video_id = data.get("data", {}).get("video_id")
        if not video_id:
            print("❌ HeyGen did not return a video ID.")
            return None

        print(f"⏳ Video requested. Polling for status (ID: {video_id})...")

        # Poll every 10 seconds for max 4 minutes
        for attempt in range(24):
            time.sleep(10)
            status_res = requests.get(HEYGEN_STATUS_URL, headers=headers, params={"video_id": video_id})
            status_res.raise_for_status()
            status_data = status_res.json()

            if status_data.get("data", {}).get("status") == "completed":
                video_url = status_data["data"].get("video_url")
                print("✅ Video ready!")
                return video_url

            elif status_data.get("data", {}).get("status") == "failed":
                print("❌ Video generation failed.")
                return None

            print(f"   Waiting... ({attempt + 1}/24)")

        print("❌ Video generation timed out.")
        return None

    except Exception as e:
        print(f"❌ Error during HeyGen video generation: {e}")
        return None


In [13]:
# STEP 7.3 (FIXED): Run CrewAI and send plain string to HeyGen

test_topic = "AI personal trainers that judge your snack choices"

# Step 1: Generate joke
print(f"\n🧠 Generating joke for topic: '{test_topic}'")
joke_result_raw = joke_crew.kickoff(inputs={"topic": test_topic})

# Extract string in case it's wrapped in CrewOutput
joke_text = str(joke_result_raw).strip().strip('"')

# Step 2: Show the joke
print(f"\n🎤 Joke from model:\n{joke_text}")

# Step 3: Generate HeyGen video
print("\n🎬 Sending joke to HeyGen for video generation...")
video_url = generate_heygen_video(joke_text)

# Step 4: Show result
if video_url:
    print(f"\n✅ Your joke video is ready: {video_url}")
else:
    print("\n❌ Something went wrong with video generation.")



🧠 Generating joke for topic: 'AI personal trainers that judge your snack choices'


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c49ed43f-510a-46f6-bf93-6dd6f2ea7162                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 8848705e-2029-4c3f-9466-2097f6ca40ff
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 8848705e-2029-4c3f-9466-2097f6ca40ff
       Status: Executing Task...
    └── 🤖 Agent: Stand-up Comedian
            Status: In Progress

# Agent: Stand-up Comedian
## Task: You will be given a topic in triple backticks: ```AI personal trainers that judge your snack choices```. Use the 'Fine-Tuned Joke Generator' tool to generate one joke based on this topic. Your output must be only the joke — no explanation, intro, or commentary.


...

🚀 Crew: crew
└── 📋 Task: 8848705e-2029-4c3f-9466-2097f6ca40ff
       Status: Executing Task...
    └── 🤖 Agent: Stand-up Comedian
            Status: ✅ Completed
        └── 🔧 Used Fine-Tuned Joke Generator (4)

🚀 Crew: crew
└── 📋 Task: 8848705e-2029-4c3f-9466-2097f6ca40ff
       Assigned to: Stand-up Comedian
       Status: ✅ Completed
    └── 🤖 Agent: Stand-up Comedian
            Status: ✅ Completed
        └── 🔧 Used Fine-Tuned Joke Generator (4)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8848705e-2029-4c3f-9466-2097f6ca40ff                                                                     │
│  Agent: Stand-up Comedian                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c49ed43f-510a-46f6-bf93-6dd6f2ea7162                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


🎤 Joke from model:
Why did the AI personal trainer bring a yoga mat to the vending machine? Because it was thirsty!

🎬 Sending joke to HeyGen for video generation...
📤 Sending joke to HeyGen...
⏳ Video requested. Polling for status (ID: 01b2326e42b5413d985dfe18fb1ae6fe)...
   Waiting... (1/24)
   Waiting... (2/24)
   Waiting... (3/24)
   Waiting... (4/24)
   Waiting... (5/24)
   Waiting... (6/24)
   Waiting... (7/24)
   Waiting... (8/24)
   Waiting... (9/24)
   Waiting... (10/24)
   Waiting... (11/24)
   Waiting... (12/24)
   Waiting... (13/24)
✅ Video ready!

✅ Your joke video is ready: https://files2.heygen.ai/aws_pacific/avatar_tmp/c7861147218245efb561558c31b63c7c/01b2326e42b5413d985dfe18fb1ae6fe.mp4?Expires=1744271111&Signature=WzI3NuRkC4e1yHWH8rMZrKXZRJBieBGjGaqbOcHzJKXGywioUig5K6-ttlsGaGX8sWjHB9NOwI8sbAWIfVRRMi7bVeZ40xu~8EVwdxGp0XXoimmhl8JgE~wk7DGYFxeIs-srtKckIVovcGoA8O43ymFskgWX-R4QUeH5r4nIh1wrkb0ITDBIcRTK-XnCnbNnhSrAroqPB-QxebLxzarLRxDbgJHkzSB0nWlwfHWwdceUJ7~YaMTDCEFM9G83DzmaM